In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import os
import pandas as pd
import ast

In [2]:

gb = []
data_path = './dataset-dev/plates_png/'
for i in os.listdir(data_path):
    if i.__contains__('GB'):
        
        gb.append(i)


In [3]:
def calculate_area(contour):
    x,y,w,h = cv2.boundingRect(contour)
    return w*h

In [4]:
def are_close(a, b) -> bool:
    """
    Compares two integers and checks if they are within 10% of each other.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        bool: True if the integers are within 10% of each other, False otherwise.
    """
    if a == 0 or b == 0:
        # Handle edge case where one or both integers are zero
        return a == b

    # Calculate the difference as a percentage of the larger value
    difference_percentage = abs(a - b) / max(abs(a), abs(b))
    return difference_percentage <= 0.3

In [ ]:
def process(img,kernel_size=5,iters=3):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_canny = cv2.Canny(img_gray, 50, 150, apertureSize=5,L2gradient=False)
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    img_dilate = cv2.dilate(img_canny, kernel, iterations=iters)

    return img_dilate

#first phase of segmentation to remove outer border present in images
def perfect_phase_1(img_path):
    img = cv2.imread(img_path)
    img_copy = np.copy(img)
    
    processed_img = process(img_copy)
    
    contours, _ = cv2.findContours(processed_img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    
    filter_contours = [ct for ct in contours if cv2.contourArea(ct) > 5000] #filter out small contours
    areas = []
    
    
    for ct in filter_contours:
        x, y, w, h = cv2.boundingRect(ct)
        #cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)
        #cv2.putText(img_copy, str(cv2.contourArea(ct)), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        areas.append(w * h)
        
    #get the two biggest contours
    top2 = []
    for i in range(3):
        largest_index = areas.index(max(areas))
        #print(max(areas))
        largest_contour = filter_contours[largest_index]    
        top2.append(largest_contour)
        
        # Remove the selected contour from the lists
        del areas[largest_index]
        del filter_contours[largest_index]
    
    
    #big = cv2.contourArea(top2[0])
    #small = cv2.contourArea(top2[1]) + cv2.contourArea(top2[2])
    big = calculate_area(top2[0])
    small = calculate_area(top2[1]) + calculate_area(top2[2]) 
    l = [big,small]
    
    #check_2_plates =min(l)/max(l)  
    
    ratio = 0
    if cv2.contourArea(top2[1]) >= cv2.contourArea(top2[2]):
        ratio = cv2.contourArea(top2[2]) / cv2.contourArea(top2[1])
    else:
        ratio = cv2.contourArea(top2[1]) / cv2.contourArea(top2[2])
    
    #print('check de 2 planches',check_2_plates)
    #print('check une planche',calculate_area(top2[1])/big)
    #print('ratio',ratio)
    rect0 = cv2.boundingRect(top2[0])
    rect1 = cv2.boundingRect(top2[1])
    rect2 = cv2.boundingRect(top2[2])
    
    cont2 = 0
    for i in range(4):
        print(rect1[i],rect2[i])
        if(are_close(rect0[i],rect1[i])):
            cont2+=1
    check_2_plates = cont2>=2
    #put the area in text on the image
    
        
    #print('width une planche',are_close_within_10_percent(w0,w1))
    #print('x une planche',are_close_within_10_percent(x0,x1))
    #print('x 2 planches',are_close_within_10_percent(x1,x2))
    #print('width 2 planches',are_close_within_10_percent(w1,w2))
    print('shape', img_copy.shape)
    cont = 0
    for i in range(4):
        print(rect0[i],rect1[i])
        if(are_close(rect0[i],rect1[i])):
            cont+=1
            
    #and ( rect1[0] != 0 and rect1[1] != 0)
    cond = cont>=2 and (rect0[0] != 0 and rect0[1] != 0 ) 
    
    print('cond',cond)
    #print([cv2.contourArea(x)   for x in top2])
    
    if calculate_area(top2[1])/big >0.7 and cond:
        print('planche complete')
        top2 = [top2[1]]
    elif check_2_plates> 0.8 and ratio > 0.7:
        top2 = top2[1:]
        print('2 planches')
       
    else:
        print('skip')
        return img_copy, [], []
        
    '''
    if small/big >0.9 and ratio > 0.7:

        top2 = top2[1:]
    else:
        top2 = [top2[1]]'''

    color =[255,255,255]

    res = []

    for ct in top2:
        cop = np.copy(img)
        stencil = np.zeros(img.shape).astype(np.uint8)

        epsilon = 0.01 * cv2.arcLength(ct, True)
        approx = cv2.approxPolyDP(ct, epsilon, True)
        hull = cv2.convexHull(approx)
        if len(hull) < 4 or len(hull)>=6 :
            continue
        cv2.drawContours(img_copy, [hull], -1, (255, 0, 0), 4)
        
        
        cv2.fillPoly(stencil, pts =[hull], color=color)

        res.append(cv2.bitwise_or(cop, ~stencil))

    return img_copy, top2, res
    

In [6]:


def is_contour_inside(contour1, contour2):
    x1, y1, w1, h1 = cv2.boundingRect(contour1)  # inside
    x2, y2, w2, h2 = cv2.boundingRect(contour2)  # outside
    if x1 >= x2 and y1 >= y2 and x1 + w1 <= x2 + w2 and y1 + h1 <= y2 + h2:
        return True

def bounding_box_to_contour(box):
    x, y, w, h = box
    contour = np.array([
        [x, y],
        [x + w, y],
        [x + w, y + h],
        [x, y + h]
    ])
    return contour

In [7]:
def second_phase(img_first_phase):
    img_copy = np.copy(img_first_phase)
    
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(process(img_copy,kernel_size=9,iters=1), connectivity=8, ltype=cv2.CV_32S)
    contours = []
    #area = img_copy.shape[0] * img_copy.shape[1]
    for i in range(1, num_labels):
        x, y, w, h, _ = stats[i]
        box = (x, y, w, h)
        contour = bounding_box_to_contour(box)
        contours.append(contour)
        
    filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > 10000]
    # get rid of small contours
    sorted_contours = sorted(filtered_contours, key=cv2.contourArea, reverse=True)
    
    draw_contours = []
    for idx,contour in enumerate(sorted_contours):
        #print(cv2.contourArea(contour))
        if idx == 0:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 0, 255), 2)
            draw_contours.append((x, y, w, h))
            #rect = cv2.minAreaRect(contour)
            #box = cv2.boxPoints(rect)
            #box = np.int0(box)
            #cv2.drawContours(img_copy,[box],0,(0,0,255),2) '''  
        else:
            #x, y, w, h = cv2.boundingRect(contour)
            res = False
            for i in range(idx):
                if is_contour_inside(contour, sorted_contours[i]):
                    res = True
                    break
            if not res: #the contour is not inside any of the previous contours
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 0, 255), 2)
                #rect = cv2.minAreaRect(contour)
                #box = cv2.boxPoints(rect)
                #box = np.int0(box)
                #cv2.drawContours(img_copy,[box],0,(0,0,255),2)
                draw_contours.append((x, y, w, h))
    return img_copy, sorted_contours,draw_contours

In [ ]:
#GB.184049.A-004.png
#GB.152368.A-016.png
#GB.282874.A-010.png
#GB.190913458.A-005.png





In [9]:
#GB.171939.A-005.png

In [10]:
#img,_,res  = perfect_phase_1(data_path+'GB.152368.A-039.png')
#GB.112704.A-012.png
#GB.125650.A-012.png
#gt[29]
img,_,res  = perfect_phase_1(data_path+"GB.184049.A-004.png")
cv2.imshow("seg", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
for img in res:
    seg,_,contours = second_phase(img)
    #print(len(contours))
    #print(contours)

    cv2.imshow("seg", seg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    


409 460
154 299
1643 1188
2704 1458
shape (3508, 2592, 3)
311 409
80 154
1761 1643
2829 2704
cond True
planche complete


In [11]:
2640939/3220032

0.820159240653509

In [12]:
data = pd.read_csv('data_dl.csv')
data

,image,label
0,GB.112704.A-011.png,"(439, 912, 1645, 1502)"
1,GB.220420.A-010.png,"(491, 726, 1554, 1761)"
2,GB.272628.A-015.png,"(190, 1824, 2046, 1191)"
3,GB.272628.A-015.png,"(203, 722, 952, 1101)"
4,GB.191506686.A-011.png,"(0, 429, 1547, 1037)"
...,...,...
1197,GB.561274.A-009.png,"(1038, 1795, 1089, 213)"
1198,GB.561274.A-009.png,"(1233, 1342, 877, 211)"
1199,GB.561274.A-009.png,"(609, 1541, 327, 449)"
1200,GB.561274.A-009.png,"(761, 1369, 425, 165)"


In [13]:
data['label'] = data['label'].apply(ast.literal_eval)
data

,image,label
0,GB.112704.A-011.png,"(439, 912, 1645, 1502)"
1,GB.220420.A-010.png,"(491, 726, 1554, 1761)"
2,GB.272628.A-015.png,"(190, 1824, 2046, 1191)"
3,GB.272628.A-015.png,"(203, 722, 952, 1101)"
4,GB.191506686.A-011.png,"(0, 429, 1547, 1037)"
...,...,...
1197,GB.561274.A-009.png,"(1038, 1795, 1089, 213)"
1198,GB.561274.A-009.png,"(1233, 1342, 877, 211)"
1199,GB.561274.A-009.png,"(609, 1541, 327, 449)"
1200,GB.561274.A-009.png,"(761, 1369, 425, 165)"


In [14]:
gt_names = data['image'].unique()
data_path = './dataset-dev/plates_png/'

In [15]:
#gt[8] #gt[16]

In [16]:
samples = pd.DataFrame(columns=['image','label'])
for name in gt_names[70:100]:
    print(name)
    img,_ ,res= perfect_phase_1(data_path+name)
    print('---')
    cv2.imshow("phase 1", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    '''for mask in res:
        img,_ ,contours= second_phase(mask)
        sample = {'image':name,'label':contours}
        samples = samples.append(sample,ignore_index=True)
        cv2.imshow("phase 1", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()'''

GB.357609.A-006.png
521 553
261 1025
1614 1487
2662 1469
shape (3508, 2592, 3)
443 521
178 261
1718 1614
2788 2662
cond True
planche complete
---
GB.152368.A-016.png
804 1381
807 1122
1146 264
2310 814
shape (3508, 2592, 3)
509 804
659 807
1906 1146
2589 2310
cond True
skip
---
GB.191223716.A-014.png
361 498
332 1006
981 272
1783 960
shape (3287, 1888, 3)
0 361
213 332
1785 981
3033 1783
cond False
skip
---
GB.317475.A-009.png
470 1216
313 780
1775 820
2698 1261
shape (3508, 2592, 3)
372 470
238 313
2053 1775
2794 2698
cond True
planche complete
---
GB.128645.A-005.png
308 1741
1046 1805
1802 308
1297 446
shape (3313, 2112, 3)
118 308
258 1046
1964 1802
2801 1297
cond False
skip
---
GB.156135.A-004.png
376 453
206 890
1642 1495
2706 1479
shape (3508, 2592, 3)
264 376
135 206
1789 1642
2828 2706
cond True
planche complete
---
GB.128645.A-004.png
118 164
970 1351
1874 1779
1421 818
shape (3350, 2096, 3)
59 118
908 970
1964 1874
1521 1421
cond True
planche complete
---
GB.174504.A-006.png

In [17]:
samples = samples.explode('label',ignore_index=True)
samples

,image,label


In [18]:
def compute_iou(rect1, rect2):
    """
    Computes the Intersection over Union (IoU) between two rectangles.
    
    Parameters:
    - rect1: Tuple (x1, y1, w1, h1) representing the first rectangle
    - rect2: Tuple (x2, y2, w2, h2) representing the second rectangle
    
    Returns:
    - IoU: The Intersection over Union (IoU) value
    """
    # Convert rectangles to (x1, y1, x2, y2)
    x1_min, y1_min, w1, h1 = rect1
    x2_min, y2_min, w2, h2 = rect2
    
    x1_max, y1_max = x1_min + w1, y1_min + h1
    x2_max, y2_max = x2_min + w2, y2_min + h2

   
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_width = max(0, inter_x_max - inter_x_min)
    inter_height = max(0, inter_y_max - inter_y_min)
    intersection_area = inter_width * inter_height
    print(f'intersection :{intersection_area}')


    area1 = w1 * h1
    area2 = w2 * h2


    union_area = area1 + area2 - intersection_area


    if union_area == 0:
        print('im here')  
        return 0
    iou = intersection_area / union_area
    return iou

In [19]:
ious = []
for i in range(len(samples)):
    ious.append(compute_iou(data['label'][i],samples['label'][i]))
ious

[]

In [20]:
name = 'GB.272628.A-015.png'
img = cv2.imread(data_path+name)
img_copy = np.copy(img)
gt_label = data[data['image'] == name]['label'].values
res_label = samples[samples['image'] == name]['label'].values
print(gt_label)
print(res_label)
for i in range(2):
    x, y, w, h = gt_label[i]
    cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 0, 255), 2)
    x, y, w, h = res_label[i]
    cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv2.imshow("img", img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

    

[(190, 1824, 2046, 1191) (203, 722, 952, 1101)]
[]


IndexError: index 0 is out of bounds for axis 0 with size 0